<a href="https://colab.research.google.com/github/zloelias/final-project/blob/master/embeddings_lenta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#! pip install tensorflow_text
#! pip install textattack['tensorflow']
#! pip install datasets --upgrade
#! pip install captum

In [ ]:
# При установке из репозитория при работе не подцепляются другие модели, работает только дефолтный bert-base-uncased

! pip install tensorflow_text
! git clone https://github.com/zloelias/TextAttack.git
! pip install /content/TextAttack[tensorflow] --use-feature=in-tree-build
! pip install datasets --upgrade
! pip install captum

     |████████████████████████████████| 4.9 MB 14.7 MB/s 
Cloning into 'TextAttack'...
remote: Enumerating objects: 20340, done.
remote: Total 20340 (delta 0), reused 0 (delta 0), pack-reused 20340
Receiving objects: 100% (20340/20340), 25.26 MiB | 29.52 MiB/s, done.
Resolving deltas: 100% (15188/15188), done.
Processing ./TextAttack
     |████████████████████████████████| 60 kB 5.7 MB/s 
     |████████████████████████████████| 322 kB 28.9 MB/s 
     |████████████████████████████████| 769 kB 81.0 MB/s 
     |████████████████████████████████| 298 kB 58.1 MB/s 
     |████████████████████████████████| 3.3 MB 66.4 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
     |████████████████████████████████| 454.3 MB 13 kB/s 
     |████████████████████████████████| 124 kB 78.0 MB/s 
     |████████████████████████████████| 462 kB 59.0 MB/s 
     |████████████████████████████████| 1.2 MB 65.5 MB/s 
     |████████████████████████████████| 4.0 MB 60.9 MB/s 
     |██████████████████████

In [ ]:
import torch
from copy import deepcopy

from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer

#from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerDeepLiftShap, InternalInfluence, LayerGradientXActivation
#from captum.attr import visualization as viz

import nltk
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('stopwords')

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:16<00:00, 29.3MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpds_i7ytg.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else: 
    device = torch.device("cpu")
    
print(device)

cuda:0


In [ ]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder, MultilingualUniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding
from textattack.shared import GensimWordEmbedding, WordEmbedding

from textattack.shared.utils import LazyLoader

from textattack.constraints.pre_transformation.max_word_index_modification import MaxWordIndexModification
from textattack.constraints.pre_transformation.max_modification_rate import MaxModificationRate

from textattack.attack_recipes.attack_recipe import AttackRecipe


In [ ]:
! wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
! wget http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
! tar xzf araneum_none_fasttextcbow_300_5_2018.tgz

--2021-12-14 13:59:23--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.250.46
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.250.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  34.2MB/s    in 47s     

2021-12-14 14:00:10 (33.8 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]

--2021-12-14 14:00:10--  http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz’


In [ ]:
from textattack.transformations import WordSwapEmbedding
from textattack.augmentation import Augmenter

from textattack.shared.utils import LazyLoader
from textattack.shared import GensimWordEmbedding, WordEmbedding

gensim = LazyLoader("gensim", globals(), "gensim")
keyed_vectors = (
    gensim.models.KeyedVectors.load('araneum_none_fasttextcbow_300_5_2018.model')
    #gensim.models.keyedvectors.Word2VecKeyedVectors.load_word2vec_format(
    #    #'https://rusvectores.org/static/models/ruscorpora_upos_skipgram_300_10_2017.bin.gz', 
    #    'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz',
    #    binary=True
    #)
)


In [ ]:
class TextFoolerJin2019RU(AttackRecipe):
    """Jin, D., Jin, Z., Zhou, J.T., & Szolovits, P. (2019).

    Is BERT Really Robust? Natural Language Attack on Text Classification and Entailment.

    https://arxiv.org/abs/1907.11932
    """

    @staticmethod
    def build(model_wrapper, max_rate, max_seq_len, language):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        #
        gensim = LazyLoader("gensim", globals(), "gensim")

        if language == 'russian':
            keyed_vectors = (
                gensim.models.KeyedVectors.load('araneum_none_fasttextcbow_300_5_2018.model')
            )
        else:
            keyed_vectors = (
                gensim.models.keyedvectors.Word2VecKeyedVectors.load_word2vec_format(
                    'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz',
                    binary=True
                )
            )
        word_embedding = GensimWordEmbedding(keyed_vectors)
        transformation = WordSwapEmbedding(max_candidates=20, embedding=word_embedding)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(language=language)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        if language == 'english':
            use_constraint = UniversalSentenceEncoder(
                threshold=0.840845057,
                metric="angular",
                compare_against_original=False,
                window_size=15,
                skip_text_shorter_than_window=True,
            )
        else:
            use_constraint = MultilingualUniversalSentenceEncoder(
                threshold=0.840845057,
                metric="angular",
                compare_against_original=False,
                window_size=15,
                skip_text_shorter_than_window=True,
            )

        constraints.append(use_constraint)

        constraints.append(MaxWordIndexModification(max_seq_len))
        constraints.append(MaxModificationRate(max_rate, min_threshold=1))

        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [ ]:
import textattack
import random
import numpy as np

experiments = {
    'word_embedding_swap': [
#        {
#            'dataset': {'name': 'kinopoisk', 'url': 'zloelias/kinopoisk-reviews'},
#            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-kinopoisk-reviews-finetuned-clf'),
#            'language': ('ru', 'rus', 'russian'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 312,
#            'columns': (['text'], 'labels'),
#        },
        {
            'dataset': {'name': 'lenta', 'url': 'zloelias/lenta-ru'},
            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-lenta-ru-finetuned-clf'),
            'language': ('ru', 'rus', 'russian'),
            'max_rate': np.linspace(0.1, 0.9, 9),
            'max_seq_len': 312,
            'columns': (['text'], 'labels'),
        },
#        {
#            'dataset': {'name': 'ag_news', 'url': 'ag_news'},
#            'model': ('bert-base-uncased', 'textattack/bert-base-uncased-ag-news'),
#            'language': ('en', 'eng', 'english'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 512,
#            'columns': (['text'], 'label'),
#        },
#        {
#            'dataset': {'name': 'rotten_tomatoes', 'url': 'rotten_tomatoes'},
#            'model': ('bert-base-uncased', 'textattack/bert-base-uncased-rotten-tomatoes'),
#            'language': ('en', 'eng', 'english'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 512,
#            'columns': (['text'], 'label'),
#        },
    ]
}

for attack_name in experiments.keys():
    for attack_data in experiments[attack_name]:
        dataset_name, dataset_url = [attack_data['dataset'][k] for k in attack_data['dataset'].keys()]
        dataset_columns = attack_data['columns']
        dataset = HuggingFaceDataset(dataset_url, None, 'test', dataset_columns=dataset_columns)

        base_model_name, model_name = attack_data['model']
        original_model = AutoModelForSequenceClassification.from_pretrained(model_name)
        original_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        model = HuggingFaceModelWrapper(original_model,original_tokenizer)

        la, lan, language = attack_data['language']
        for max_rate in attack_data['max_rate']:
            experiment_result_dir = f'/content/drive/MyDrive/Colab Notebooks/final-project/experiments/{attack_name}/{dataset_name}/{max_rate}/'
            print(experiment_result_dir)
            attack = TextFoolerJin2019RU.build(model, max_rate, attack_data['max_seq_len'], language)

            attack_args = textattack.AttackArgs(
                num_examples=200, 
                log_to_csv=experiment_result_dir + "log.csv", 
                checkpoint_interval=50, 
                checkpoint_dir=experiment_result_dir + "checkpoints", 
                disable_stdout=True
            )
            attacker = textattack.Attacker(attack, dataset, attack_args)
            attacker.attack_dataset()
        #print(attack_data)
        #print(model_name, model_url)

Using custom data configuration zloelias___lenta-ru-dc5084b443d90e4e


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/26.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/240M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

[Succeeded / Failed / Skipped / Total] 7 / 43 / 0 / 50:  25%|██▌       | 50/200 [11:52<35:37, 14.25s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/word_embedding_swap/lenta/0.9/checkpoints/1639510736057.ta.chkpt" at 2021-12-14 19:38:56 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 80 / 1 / 100:  50%|█████     | 100/200 [22:14<22:14, 13.35s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/word_embedding_swap/lenta/0.9/checkpoints/1639511358164.ta.chkpt" at 2021-12-14 19:49:18 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 119 / 1 / 150:  75%|███████▌  | 150/200 [33:00<11:00, 13.20s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/word_embedding_swap/lenta/0.9/checkpoints/1639512003796.ta.chkpt" at 2021-12-14 20:00:03 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 41 / 158 / 1 / 200: 100%|██████████| 200/200 [41:53<00:00, 12.57s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/word_embedding_swap/lenta/0.9/checkpoints/1639512536613.ta.chkpt" at 2021-12-14 20:08:56 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 41 / 158 / 1 / 200: 100%|██████████| 200/200 [41:53<00:00, 12.57s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 41     |
| Number of failed attacks:     | 158    |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 99.5%  |
| Accuracy under attack:        | 79.0%  |
| Attack success rate:          | 20.6%  |
| Average perturbed word %:     | 9.66%  |
| Average num. words per input: | 187.64 |
| Avg num queries:              | 832.6  |
+-------------------------------+--------+
